# **Submitted by:**
#### Kiran Gupta
#### Parama Bhattacharya

### Problem Statement:
Our model should be able to identify gestures which correspond to specific commands in a smart TV. There are mentioned below:
#####	***Thumbs up:  Increase the volume***
#####	***Thumbs down: Decrease the volume***
#####	***Left swipe: 'Jump' backwards 10 seconds***
#####	***Right swipe: 'Jump' forward 10 seconds*** 
#####	***Stop: Pause the movie*** 
Each video (typically 2-3 seconds long) is divided into a sequence of 30 frames(images). These videos have been recorded by various people performing one of the five gestures in front of a webcam - similar to what the smart TV will use.  
Each video is a sequence of 30 frames (or images). All images in a particular video subfolder have the same dimensions but different videos may have different dimensions. Specifically, videos have two types of dimensions - either 360x360 or 120x160 (depending on the webcam used to record the videos). 
Hence, we need to do some pre-processing to standardise the videos. We have done resizing of the frames to a shape of 120 X 120.


# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
from scipy.misc import imread, imresize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import datetime
import os

In [ ]:
import cv2
import matplotlib.pyplot as plt
% matplotlib inline

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
DATASET_PATH = 'Project_data'

In [ ]:
train_doc = np.random.permutation(open(DATASET_PATH + '/' + 'train.csv').readlines())
val_doc = np.random.permutation(open(DATASET_PATH + '/' + 'val.csv').readlines())
batch_size = 24 #experiment with the batch size

Parameter Initialization

In [ ]:
# input image dimensions
img_height = 160  # X dimension of the image
img_width = 160   # Y dimesnion of the image

img_frames = 30  # Number of image frames per video

# number of channels
# For grayscale use 1 value and for color images use 3 (R,G,B channels)
img_channel = 3

## Number of output classes
## eg: In my case I wanted to predict 5 types of gestures (Thumbs up, Thumbs down, Left swipe, Right Swipe, Stop)
nb_classes = 5

In [ ]:
# define the function to normalize the image
def normalize_img(image):
    norm_image = image - np.min(image)/np.max(image) - np.min(image)
    return norm_image

In [ ]:
# For transfer Learning with mobilenet

img_frames = 16  # Number of image frames per video
batch_size = 8

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, img_frames)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            #batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_data = np.zeros((batch_size, img_frames, img_height, img_width, img_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    # Need to resize the data so that they all are of same size.
                    resized_image = imresize(image, (img_height, img_width, img_channel))
                    # Normalization of the images is required
                    normalized_image = normalize_img(resized_image)
                    batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        batch_remaining = len(t)%batch_size
        if(batch_remaining != 0):
            batch_data = np.zeros((len(t)%batch_size, img_frames, img_height, img_width, img_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # Need to resize the data so that they all are of same size.
                    resized_image = imresize(image, (img_height, img_width, img_channel))
                    # Normalization of the images is required                 
                    normalized_image = normalize_img(resized_image)
                    batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do		

## Augmented Data Generator
In this generator, unlike the previous one where we are performing only
1. Resizing of the Image
2. Normalization of the resized image

Additionally, the following transformations are carried out for the purpose of augmentation:
1. Accepting a batch of images used for training.
2. Taking the batch and applying a series of random transformations to each image in the batch including random rotation
3. Replacing the original batch with the new, randomly transformed batch.

In [ ]:
def augmented_generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #create a list of image numbers you want to use for a particular video
    img_idx = [x for x in range(0, img_frames)]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size, img_frames, img_height, img_width, img_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    # Need to perform affine transformation for understanding the relation between 2 images
                    # warpAffine( src, warp_dst, warp_mat, warp_dst.size() ), here src is the input image, warp_dst is the output image, warp_mat is the transformation and warp_dst.size() is the desired size of the image
                    M = cv2.getRotationMatrix2D((img_width//2, img_height//2), np.random.randint(-30, 30), 1.0)
                    transformed_image = cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))
                    
                    # Need to resize the data so that they all are of same size.
                    resized_image = imresize(transformed_image, (img_height, img_width, img_channel))
                    # Normalization of the images is required
                    normalized_image = normalize_img(resized_image)                    
                    
                    batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image
     
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        batch_remaining = len(t)%batch_size
        if(batch_remaining != 0):
            batch_data = np.zeros((len(t)%batch_size, img_frames, img_height, img_width, img_channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((len(t)%batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(len(t)%batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate over the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    # Need to perform affine transformation for understanding the relation between 2 images
                    # warpAffine( src, warp_dst, warp_mat, warp_dst.size() ), here src is the input image, warp_dst is the output image, warp_mat is the transformation and warp_dst.size() is the desired size of the image
                    M = cv2.getRotationMatrix2D((img_width//2, img_height//2), np.random.randint(-30, 30), 1.0)
                    transformed_image = cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))

                    # Need to resize the data so that they all are of same size.
                    resized_image = imresize(transformed_image, (img_height, img_width, img_channel))
                    # Normalization of the images is required
                    normalized_image = normalize_img(resized_image)                    
                    
                    batch_data[folder,idx,:,:,0] = normalized_image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalized_image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalized_image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do	

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20   # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
#Define all the library
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Dropout, BatchNormalization, Activation
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.optimizers import Adam
from keras.applications import mobilenet

# Conv2D + RNN

In [ ]:
#write your model here
img_filter = [8,16,32,64]
img_dense = [128,64]
dropout = [0.5, 0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(TimeDistributed(Conv2D(img_filter[0], kernel_size=(3,3), padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 2
model.add(TimeDistributed(Conv2D(img_filter[1], (3, 3), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 3
model.add(TimeDistributed(Conv2D(img_filter[1], (2, 2), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 4
model.add(TimeDistributed(Conv2D(img_filter[1], (2, 2), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 100, 100, 8)   224       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 100, 100, 8)   32        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 50, 50, 8)     0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 50, 50, 16)    1168      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 50, 50, 16)    64        
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 25, 25, 16)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 25, 25, 16)    1040      
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
batch_size = 10
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


66/67 [============================>.] - ETA: 2s - loss: 6.6969 - categorical_accuracy: 0.2788

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 192s 3s/step - loss: 6.6849 - categorical_accuracy: 0.2796 - val_loss: 3.4033 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-08-1411_39_44.901874/model-00001-6.69321-0.27903-3.40325-0.30000.h5
Epoch 2/10
67/67 [==============================] - 76s 1s/step - loss: 4.2941 - categorical_accuracy: 0.3805 - val_loss: 1.4490 - val_categorical_accuracy: 0.5000

Epoch 00002: saving model to model_init_2020-08-1411_39_44.901874/model-00002-4.33782-0.37406-1.44904-0.50000.h5
Epoch 3/10
67/67 [==============================] - 77s 1s/step - loss: 2.9259 - categorical_accuracy: 0.3956 - val_loss: 2.2104 - val_categorical_accuracy: 0.5200

Epoch 00003: saving model to model_init_2020-08-1411_39_44.901874/model-00003-2.86626-0.39970-2.21038-0.52000.h5
Epoch 4/10
67/67 [==============================] - 77s 1s/step - loss: 2.9940 - categorical_accuracy: 0.4139 - val_loss: 1.5903 - val_categorical_accuracy: 0.4900

Epoch 00004

**Observation**

*   Training Accuracy: 47.7%
*   Validation Accuracy: 45%
*   Learning Rate: 0.0005000000237487257
*   Filter Size: 3 X 3
*   Batch Size: 10
*   Epoch: 10



In [ ]:
batch_size = 10
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.3634 - categorical_accuracy: 0.4615

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:38: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 76s 1s/step - loss: 1.3631 - categorical_accuracy: 0.4572 - val_loss: 1.1615 - val_categorical_accuracy: 0.5200

Epoch 00001: saving model to model_init_2020-08-1411_39_44.901874/model-00001-1.36109-0.45852-1.16153-0.52000.h5
Epoch 2/30
67/67 [==============================] - 76s 1s/step - loss: 1.3411 - categorical_accuracy: 0.4984 - val_loss: 1.6216 - val_categorical_accuracy: 0.4300

Epoch 00002: saving model to model_init_2020-08-1411_39_44.901874/model-00002-1.34807-0.49321-1.62161-0.43000.h5
Epoch 3/30
67/67 [==============================] - 75s 1s/step - loss: 1.2677 - categorical_accuracy: 0.5035 - val_loss: 1.1324 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_init_2020-08-1411_39_44.901874/model-00003-1.26956-0.50528-1.13241-0.58000.h5
Epoch 4/30
67/67 [==============================] - 75s 1s/step - loss: 1.2470 - categorical_accuracy: 0.5404 - val_loss: 1.2066 - val_categorical_accuracy: 0.5700

Epoch 00004:

**Observation**

*   Training Accuracy: 72.3%
*   Validation Accuracy: 69%
*   Learning Rate: 4.882812731921149e-07
*   Filter Size: 3 X 3
*   Batch Size: 10
*   Epoch: 30

**Conclusion**
For the Conv2D we can see that as and when the number of epochs increases, the accuracy increases. But we had to restrict to a epoch of size 30 due to memory contraints.

# Conv3D + RNN

**Model 1 - No Data Augmentation and Adam optimizer with
            Image size 160 X 160, Batch Size 24 and Epoch 10**

In [ ]:
#write your model here
img_filter = [8,16,32,64]
img_dense = [256,128]
dropout = [0.5, 0.5]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], (3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], (1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], (1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 160, 160, 8)   3008      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 160, 160, 8)   0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 160, 160, 8)   32        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 80, 80, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 80, 80, 16)    3472      
_________________________________________________________________
activation_2 (Activation)    (None, 15, 80, 80, 16)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 80, 80, 16)    64        
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 24
Source path =  Project_data/train ; batch size = 24
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 204s 7s/step - loss: 2.0363 - categorical_accuracy: 0.3561 - val_loss: 2.1340 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_conv3D_2020-08-1608_38_46.350670/model-00001-2.03345-0.35897-2.13400-0.28000.h5
Epoch 2/10
28/28 [==============================] - 87s 3s/step - loss: 1.4625 - categorical_accuracy: 0.5060 - val_loss: 1.1877 - val_categorical_accuracy: 0.4900

Epoch 00002: saving model to model_conv3D_2020-08-1608_38_46.350670/model-00002-1.45387-0.50830-1.18769-0.49000.h5
Epoch 3/10
28/28 [==============================] - 91s 3s/step - loss: 1.2393 - categorical_accuracy: 0.5307 - val_loss: 1.0729 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_conv3D_2020-08-1608_38_46.350670/model-00003-1.23598-0.53243-1.07291-0.58000.h5
Epoch 4/10
28/28 [==============================] - 92s 3s/step - loss: 1.0435 - categorical_accuracy: 0.6107 - val_loss: 1.0124 - val_categorical_accuracy: 0.5700

Epoch

**Observation**

*   Training Accuracy: 74.6%
*   Validation Accuracy: 63%
*   Learning Rate: 0.001
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 10
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,702,821

**Model 2 - No Data Augmentation and Adam optimizer with
            Image size 120 X 120, Batch Size 24 and Epoch 10**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.01
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_14 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model2_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 24
Source path =  Project_data/train ; batch size = 24
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/28 [==>...........................] - ETA: 1:54 - loss: 2.0010 - categorical_accuracy: 0.2778

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 88s 3s/step - loss: 1.7393 - categorical_accuracy: 0.3923 - val_loss: 13.1037 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00001-1.74317-0.39216-13.10369-0.18000.h5
Epoch 2/10
28/28 [==============================] - 78s 3s/step - loss: 1.3826 - categorical_accuracy: 0.3965 - val_loss: 7.8441 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00002-1.38217-0.39819-7.84411-0.23000.h5
Epoch 3/10
28/28 [==============================] - 80s 3s/step - loss: 1.1805 - categorical_accuracy: 0.4824 - val_loss: 2.2178 - val_categorical_accuracy: 0.3800

Epoch 00003: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00003-1.17894-0.48265-2.21778-0.38000.h5
Epoch 4/10
28/28 [==============================] - 80s 3s/step - loss: 1.1096 - categorical_accuracy: 0.5416 - val_loss: 2.0047 - val_categorical_accuracy: 0.3800

E

**Observation**

*   Training Accuracy: 74%
*   Validation Accuracy: 69%
*   Learning Rate: 0.005
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 10
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,752,901

**Model 3 - No Data Augmentation and Adam optimizer with
            Image size 120 X 120, Batch Size 24 and Epoch 30**

In [ ]:
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/28 [==>...........................] - ETA: 1:33 - loss: 0.3524 - categorical_accuracy: 0.8889

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 87s 3s/step - loss: 0.3746 - categorical_accuracy: 0.8634 - val_loss: 0.7911 - val_categorical_accuracy: 0.7200

Epoch 00001: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00001-0.37419-0.86425-0.79107-0.72000.h5
Epoch 2/30
28/28 [==============================] - 80s 3s/step - loss: 0.3415 - categorical_accuracy: 0.8866 - val_loss: 0.7362 - val_categorical_accuracy: 0.7000

Epoch 00002: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00002-0.34073-0.88688-0.73618-0.70000.h5
Epoch 3/30
28/28 [==============================] - 80s 3s/step - loss: 0.2515 - categorical_accuracy: 0.9134 - val_loss: 1.0023 - val_categorical_accuracy: 0.6500

Epoch 00003: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00003-0.25061-0.91403-1.00226-0.65000.h5
Epoch 4/30
28/28 [==============================] - 80s 3s/step - loss: 0.2747 - categorical_accuracy: 0.9024 - val_loss: 0.8226 - val_categorical_accuracy: 0.7100

Epo

**Observation**

*   Training Accuracy: 96.8%
*   Validation Accuracy: 82%
*   Learning Rate: 0.0006
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,752,901

**Model 4 - No Data Augmentation and Adam optimizer with
            Image size 120 X 120, Batch Size 32 and Epoch 30**

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 86s 3s/step - loss: 0.0540 - categorical_accuracy: 0.9828 - val_loss: 0.5753 - val_categorical_accuracy: 0.8100

Epoch 00001: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00001-0.05310-0.98341-0.57532-0.81000.h5
Epoch 2/30
28/28 [==============================] - 81s 3s/step - loss: 0.0595 - categorical_accuracy: 0.9866 - val_loss: 0.5052 - val_categorical_accuracy: 0.8200

Epoch 00002: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00002-0.05943-0.98643-0.50518-0.82000.h5
Epoch 3/30
28/28 [==============================] - 81s 3s/step - loss: 0.0513 - categorical_accuracy: 0.9857 - val_loss: 0.6851 - val_categorical_accuracy: 0.7800

Epoch 00003: saving model to model2_conv3D_2020-08-1608_38_46.350670/model-00003-0.05063-0.98643-0.68508-0.78000.h5
Epoch 4/30
28/28 [==============================] - 80s 3s/step - loss: 0.0523 - categorical_accuracy: 0.9955 - val_loss: 0.4138 - val_categorical_accuracy: 0.8400

Epo

**Observation**

*   Training Accuracy: 98%
*   Validation Accuracy: 87%
*   Learning Rate: 1.220703097715159e-06
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 32
*   Epoch: 10
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,752,901

**Model 5 - No Data Augmentation and Adam optimizer with
            Filter Size of (3,3,3), Image size 120 X 120, Batch Size 32 and Epoch 30**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(3, 3, 3), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.01
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_18 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model4_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  Project_data/train ; batch size = 32
; batch size = 32
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 2:11 - loss: 2.3826 - categorical_accuracy: 0.2031

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 92s 4s/step - loss: 1.7611 - categorical_accuracy: 0.3752 - val_loss: 12.4109 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model4_conv3D_2020-08-1608_38_46.350670/model-00001-1.76100-0.37557-12.41093-0.23000.h5
Epoch 2/30
21/21 [==============================] - 76s 4s/step - loss: 1.2326 - categorical_accuracy: 0.5014 - val_loss: 8.6929 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model4_conv3D_2020-08-1608_38_46.350670/model-00002-1.23323-0.50226-8.69289-0.23000.h5
Epoch 3/30
21/21 [==============================] - 80s 4s/step - loss: 1.0957 - categorical_accuracy: 0.5474 - val_loss: 5.5931 - val_categorical_accuracy: 0.2400

Epoch 00003: saving model to model4_conv3D_2020-08-1608_38_46.350670/model-00003-1.09981-0.54600-5.59310-0.24000.h5
Epoch 4/30
21/21 [==============================] - 79s 4s/step - loss: 0.9683 - categorical_accuracy: 0.6095 - val_loss: 1.8976 - val_categorical_accuracy: 0.3900

E

**Observation**

*   Training Accuracy: 85%
*   Validation Accuracy: 74%
*   Learning Rate: 0.005
*   Filter Size: 3 X 3, 3 X 3
*   Batch Size: 32
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,932,517

**Inference**

So changing filter size has reduced the accuracy. So will stick to the filter size of 5 X 5 and 3 X 3. Let me change the strides and see whether it makes any difference.

**Model 6 - No Data Augmentation and Adam optimizer with
            Filter Size of (5,5,5) followed by(3,3,3), Image size 120 X 120, Strides (2,2) and Batch Size 32 and Epoch 30**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), strides=(1,1,1), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), strides=(1,1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), strides=(1,1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), strides=(1,1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2,2,2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.01
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model5_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 24
Source path =  Project_data/train ; batch size = 24
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 193s 7s/step - loss: 1.8853 - categorical_accuracy: 0.3695 - val_loss: 10.2888 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model5_conv3D_2020-08-1612_43_33.908554/model-00001-1.89107-0.36652-10.28880-0.25000.h5
Epoch 2/10
28/28 [==============================] - 75s 3s/step - loss: 1.1721 - categorical_accuracy: 0.5286 - val_loss: 4.4683 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model5_conv3D_2020-08-1612_43_33.908554/model-00002-1.16659-0.52941-4.46833-0.28000.h5
Epoch 3/10
28/28 [==============================] - 77s 3s/step - loss: 0.8941 - categorical_accuracy: 0.6631 - val_loss: 7.0511 - val_categorical_accuracy: 0.2500

Epoch 00003: saving model to model5_conv3D_2020-08-1612_43_33.908554/model-00003-0.89776-0.66214-7.05108-0.25000.h5
Epoch 4/10
28/28 [==============================] - 79s 3s/step - loss: 0.8662 - categorical_accuracy: 0.6473 - val_loss: 4.1422 - val_categorical_accuracy: 0.2700



**Model 7 - No Data Augmentation and Adadelta optimizer with
            Filter Size of (5,5,5) followed by(3,3,3), Image size 120 X 120 and Batch Size 32 and Epoch 30**

In [ ]:
from keras import optimizers
import keras

learning_rate = 0.001
model.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model6_conv3d' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 24
Source path =  Project_data/trainEpoch 1/30
 ; batch size = 24


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


28/28 [==============================] - 194s 7s/step - loss: 1.6129 - categorical_accuracy: 0.4065 - val_loss: 1.1122 - val_categorical_accuracy: 0.5900

Epoch 00001: saving model to model6_conv3d_2020-08-1613_15_57.168437/model-00001-1.61545-0.40573-1.11225-0.59000.h5
Epoch 2/30
28/28 [==============================] - 76s 3s/step - loss: 1.1210 - categorical_accuracy: 0.5686 - val_loss: 1.2539 - val_categorical_accuracy: 0.5400

Epoch 00002: saving model to model6_conv3d_2020-08-1613_15_57.168437/model-00002-1.12910-0.56561-1.25388-0.54000.h5
Epoch 3/30
28/28 [==============================] - 80s 3s/step - loss: 0.8876 - categorical_accuracy: 0.6367 - val_loss: 1.4679 - val_categorical_accuracy: 0.4200

Epoch 00003: saving model to model6_conv3d_2020-08-1613_15_57.168437/model-00003-0.88892-0.63801-1.46790-0.42000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/30
28/28 [==============================] - 80s 3s/step - loss: 0.7556 - categorical_accuracy: 0

**Observation**

*   Training Accuracy: 95%
*   Validation Accuracy: 82%
*   Learning Rate: 0.008
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 30
*   Optimizer: Adadelta
*   Augmentation: None
*   Trainable Parameters: 1,752,901

**Inference**

Accuracy was much higher when we used Adam optimizer. Also the learning rate has increased with Adadelta. So from henceforth we will be using the Adam optimizer for the rest of the models

**Model 8 - Data Augmentation and Adam optimizer having image size 120 X 120 with Batch Size 32 and Epoch 10**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_10 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 32)    128       
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = augmented_generator(train_path, train_doc, batch_size)
val_generator = augmented_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model8_conv3d' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 212s 10s/step - loss: 1.8413 - categorical_accuracy: 0.3479 - val_loss: 3.0295 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model8_conv3d_2020-08-1616_56_15.829707/model-00001-1.84137-0.34842-3.02946-0.28000.h5
Epoch 2/10
21/21 [==============================] - 86s 4s/step - loss: 1.3918 - categorical_accuracy: 0.4582 - val_loss: 3.5258 - val_categorical_accuracy: 0.2900

Epoch 00002: saving model to model8_conv3d_2020-08-1616_56_15.829707/model-00002-1.39263-0.45852-3.52581-0.29000.h5
Epoch 3/10
21/21 [==============================] - 90s 4s/step - loss: 1.2381 - categorical_accuracy: 0.5343 - val_loss: 2.0619 - val_categorical_accuracy: 0.4400

Epoch 00003: saving model to model8_conv3d_2020-08-1616_56_15.829707/model-00003-1.24554-0.52941-2.06190-0.44000.h5
Epoch 4/10
21/21 [==============================] - 89s 4s/step - loss: 1.0863 - categorical_accuracy: 0.5799 - val_loss: 1.4501 - val_categorical_accuracy: 0.5300

E

**Observation**

*   Training Accuracy: 74.7%
*   Validation Accuracy: 68%
*   Learning Rate: 0.001
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 10
*   Optimizer: Adam
*   Augmentation: Yes
*   Trainable Parameters: 1,752,901

**Model 9 - Data Augmentation and Adam optimizer having image size 100 X 100 with Batch Size 32 and Epoch 10**

In [ ]:
#write your model here
img_height = 100
img_width = 100
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 100, 100, 16)  6016      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 100, 100, 16)  0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 100, 100, 16)  64        
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 50, 50, 16)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 50, 50, 32)    13856     
_________________________________________________________________
activation_14 (Activation)   (None, 15, 50, 50, 32)    0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 50, 50, 32)    128       
__________

In [ ]:
train_generator = augmented_generator(train_path, train_doc, batch_size)
val_generator = augmented_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model9_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 2:19 - loss: 2.1513 - categorical_accuracy: 0.2969

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 101s 5s/step - loss: 1.9180 - categorical_accuracy: 0.3555 - val_loss: 3.5765 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model9_conv3D_2020-08-1616_56_15.829707/model-00001-1.92350-0.35445-3.57652-0.28000.h5
Epoch 2/10
21/21 [==============================] - 83s 4s/step - loss: 1.3725 - categorical_accuracy: 0.4582 - val_loss: 3.3277 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model9_conv3D_2020-08-1616_56_15.829707/model-00002-1.37615-0.45852-3.32771-0.25000.h5
Epoch 3/10
21/21 [==============================] - 88s 4s/step - loss: 1.2613 - categorical_accuracy: 0.5109 - val_loss: 1.8199 - val_categorical_accuracy: 0.4500

Epoch 00003: saving model to model9_conv3D_2020-08-1616_56_15.829707/model-00003-1.25664-0.51131-1.81986-0.45000.h5
Epoch 4/10
21/21 [==============================] - 89s 4s/step - loss: 1.2030 - categorical_accuracy: 0.5554 - val_loss: 2.1319 - val_categorical_accuracy: 0.4200

Ep

**Observation**

*   Training Accuracy: 66%
*   Validation Accuracy: 59%
*   Learning Rate: 0.0005
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 32
*   Epoch: 10
*   Optimizer: Adam
*   Augmentation: Yes
*   Trainable Parameters: 1,326,917

**Inference**

Both training aand validation acuracy has significantly reduced on reducing the image size from 120 X 120 to 100 X 100. Also the learning rate has become half on reducing the image size. We will be continuing our further experimentation on a image of size 120 X 120.

**Model 10 - Data Augmentation and Adam optimizer having image size 120 X 120 with Batch Size 32 and Epoch 30**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_17 (Conv3D)           (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_17 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_17 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_18 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_18 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = augmented_generator(train_path, train_doc, batch_size)
val_generator = augmented_generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_10_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/21 [=>............................] - ETA: 2:09 - loss: 2.2018 - categorical_accuracy: 0.2031

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:57: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:75: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 102s 5s/step - loss: 1.8575 - categorical_accuracy: 0.3579 - val_loss: 4.7349 - val_categorical_accuracy: 0.2800

Epoch 00001: saving model to model_10_conv3D_2020-08-1616_56_15.829707/model-00001-1.86456-0.35747-4.73492-0.28000.h5
Epoch 2/30
21/21 [==============================] - 88s 4s/step - loss: 1.4701 - categorical_accuracy: 0.4582 - val_loss: 4.0973 - val_categorical_accuracy: 0.3100

Epoch 00002: saving model to model_10_conv3D_2020-08-1616_56_15.829707/model-00002-1.47221-0.45852-4.09734-0.31000.h5
Epoch 3/30
21/21 [==============================] - 91s 4s/step - loss: 1.1895 - categorical_accuracy: 0.5410 - val_loss: 1.5951 - val_categorical_accuracy: 0.4600

Epoch 00003: saving model to model_10_conv3D_2020-08-1616_56_15.829707/model-00003-1.18858-0.54299-1.59511-0.46000.h5
Epoch 4/30
21/21 [==============================] - 91s 4s/step - loss: 1.0307 - categorical_accuracy: 0.5969 - val_loss: 1.1387 - val_categorical_accuracy: 0.55

**Observation**

*   Training Accuracy: 91%
*   Validation Accuracy: 82%
*   Learning Rate: 3.125000148429535e-05
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 32
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,752,901

**Inference**


*   Training accuracy diminished from ***98% (without data augmentation)*** to ***91% (with data augmentation)***
*   Validation accuracy diminished from ***87% (without data augmentation)*** to ***82% (with data augmentation)***

Hence we will continue our further experiments without any data augmentations


**Model 11 - Data Augmentation and Adam optimizer with increased number of layers [double of the earlier ones], having image size 120 X 120 with Batch Size 32 and Epoch 30**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = [256,128]
dropout = [0.5,0.25]

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense[0],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense[1],activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_21 (Conv3D)           (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_21 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 30, 120, 120, 16)  32016     
_________________________________________________________________
activation_22 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_32 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 15, 60, 60, 16)    0         
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_11_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Epoch 1/20
Project_data/train ; batch size = 32


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 187s 9s/step - loss: 1.7814 - categorical_accuracy: 0.3683 - val_loss: 11.2991 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00001-1.77822-0.36802-11.29906-0.23000.h5
Epoch 2/20
21/21 [==============================] - 132s 6s/step - loss: 1.1074 - categorical_accuracy: 0.5790 - val_loss: 1.3951 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00002-1.10593-0.57919-1.39512-0.40000.h5
Epoch 3/20
21/21 [==============================] - 131s 6s/step - loss: 0.8603 - categorical_accuracy: 0.6540 - val_loss: 1.1432 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00003-0.86263-0.65460-1.14320-0.54000.h5
Epoch 4/20
21/21 [==============================] - 132s 6s/step - loss: 0.6877 - categorical_accuracy: 0.7391 - val_loss: 1.1439 - val_categorical_accuracy:

**Observation**

*   Training Accuracy: 98%
*   Validation Accuracy: 72%
*   Learning Rate: 0.0005
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 32
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 1,997,589

**Inference**

By doubling the number of hidden layers, the training accuracy reamained same but validation accuracy has diminished from **82%** to **72%**. Hence there is no need to increase the number of layers. Moreover we can see that the data is overfitting

**Model 12 - Data Augmentation and Adam optimizer with reduced parameters, having image size 120 X 120 with Batch Size 32 and Epoch 30**

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128]
img_dense = 128
dropout = [0.5,0.25]


# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(Conv3D(img_filter[0], kernel_size=(5, 5, 5), padding='same', input_shape=input_shape))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 2
model.add(Conv3D(img_filter[1], kernel_size=(3, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 3
model.add(Conv3D(img_filter[2], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

# Layer 4
model.add(Conv3D(img_filter[3], kernel_size=(1, 3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

#Flatten the layers
model.add(Flatten())

#Fully Connected Layer
model.add(Dense(img_dense,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[0]))

model.add(Dense(img_dense,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(dropout[1]))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
learning_rate = 0.001
optimiser = Adam(learning_rate)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 16)  6016      
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_41 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_25 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_42 (Batc (None, 15, 60, 60, 32)    128       
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_11_conv3D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 1:35 - loss: 2.3973 - categorical_accuracy: 0.2292

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 89s 4s/step - loss: 1.8088 - categorical_accuracy: 0.3538 - val_loss: 1.6505 - val_categorical_accuracy: 0.3400

Epoch 00001: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00001-1.81116-0.35445-1.65050-0.34000.h5
Epoch 2/30
21/21 [==============================] - 73s 3s/step - loss: 1.2527 - categorical_accuracy: 0.5040 - val_loss: 1.0908 - val_categorical_accuracy: 0.6000

Epoch 00002: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00002-1.25331-0.50377-1.09079-0.60000.h5
Epoch 3/30
21/21 [==============================] - 77s 4s/step - loss: 0.9847 - categorical_accuracy: 0.6207 - val_loss: 1.2272 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_11_conv3D_2020-08-1616_56_15.829707/model-00003-0.98429-0.62142-1.22723-0.49000.h5
Epoch 4/30
21/21 [==============================] - 78s 4s/step - loss: 0.9014 - categorical_accuracy: 0.6173 - val_loss: 1.0721 - val_categorical_accuracy: 0.510

**Observation**

*   Training Accuracy: 98%
*   Validation Accuracy: 80%
*   Learning Rate: 1.5625000742147677e-05
*   Filter Size: 5 X 5, 3 X 3
*   Batch Size: 24
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 933,317

**Inference**

***Reducing the number of trainable parameters*** from **1752901** to **933317** actually ***reduced the accuracy*** from **87%** to **80%**. So let us stick to our original model having **1752901** parameters.

# CNN + LSTM

In [ ]:
#write your model here
img_height = 120
img_width = 120
img_filter = [16,32,64,128,256]
img_dense = 64
lstm_cells = 64
dropout = 0.25

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

# Input Layer
model = Sequential()
model.add(TimeDistributed(Conv2D(img_filter[0], kernel_size=(3,3), padding='same', activation='relu'), input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 2
model.add(TimeDistributed(Conv2D(img_filter[1], (3, 3), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 3
model.add(TimeDistributed(Conv2D(img_filter[2], (2, 2), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 4
model.add(TimeDistributed(Conv2D(img_filter[3], (2, 2), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

# Layer 5
model.add(TimeDistributed(Conv2D(img_filter[4], (2, 2), padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))

#Flatten the layers
model.add(TimeDistributed(Flatten()))

# LSTM Layer
model.add(LSTM(lstm_cells))
model.add(Dropout(dropout))

#Fully Connected Layer
model.add(Dense(img_dense,activation='relu'))
model.add(Dropout(dropout))

#softmax layer
model.add(Dense(nb_classes,activation='softmax'))

In [ ]:
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 30, 30, 64)    8256      
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_LSTM' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 32
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 32
Source path =  Project_data/train ; batch size = 32
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/21 [===>..........................] - ETA: 1:34 - loss: 1.5596 - categorical_accuracy: 0.3750

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


21/21 [==============================] - 90s 4s/step - loss: 1.4490 - categorical_accuracy: 0.3603 - val_loss: 1.2505 - val_categorical_accuracy: 0.5400

Epoch 00001: saving model to model_LSTM_2020-08-1616_56_15.829707/model-00001-1.44938-0.36048-1.25054-0.54000.h5
Epoch 2/30
21/21 [==============================] - 71s 3s/step - loss: 1.2463 - categorical_accuracy: 0.4644 - val_loss: 1.1506 - val_categorical_accuracy: 0.5500

Epoch 00002: saving model to model_LSTM_2020-08-1616_56_15.829707/model-00002-1.24854-0.46305-1.15064-0.55000.h5
Epoch 3/30
21/21 [==============================] - 78s 4s/step - loss: 1.1444 - categorical_accuracy: 0.5347 - val_loss: 1.1046 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_LSTM_2020-08-1616_56_15.829707/model-00003-1.14498-0.53544-1.10463-0.55000.h5
Epoch 4/30
21/21 [==============================] - 77s 4s/step - loss: 1.1179 - categorical_accuracy: 0.5494 - val_loss: 1.0803 - val_categorical_accuracy: 0.6200

Epoch 00004:

**Observation**

*   Training Accuracy: 97%
*   Validation Accuracy: 73%
*   Learning Rate: 0.000125
*   Batch Size: 32
*   Epoch: 30
*   Optimizer: Adam
*   Augmentation: None
*   Trainable Parameters: 789,509

**Inference**

It is overfitting

# Transfer Learning with Mobilenet

In [ ]:
#write your model here
img_height = 120
img_width = 120
lstm_layer = 64
img_dense = 64
dropout = 0.25
img_frames = 16  # Number of image frames per video
batch_size = 8

# Input
input_shape=(img_frames,img_height,img_width,img_channel)

#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = mobilenet.MobileNet(weights='imagenet', include_top=False)
model = Sequential()
model.add(TimeDistributed(base_model, input_shape=input_shape))

for layer in model.layers:
	layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

#LSTM layers
model.add(LSTM(lstm_layer))
model.add(Dropout(dropout))

#Fully Connected Layer
model.add(Dense(img_dense,activation='relu'))
model.add(Dropout(dropout))

#softmax layer
model.add(Dense(nb_classes, activation='softmax'))

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/keras_applications/mobilenet.py:206: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [ ]:
optimiser = Adam()
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_17 (TimeDis (None, 16, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_18 (TimeDis (None, 16, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_19 (TimeDis (None, 16, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 16, 1024)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                278784    
_________________________________________________________________
dropout_8 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
__________

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_TL_Mobilenet' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [ ]:
batch_size = 8
num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 8Source path =  Project_data/train ; batch size = 8
Epoch 1/20



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


82/83 [============================>.] - ETA: 0s - loss: 1.5744 - categorical_accuracy: 0.2927

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


83/83 [==============================] - 44s 535ms/step - loss: 1.5702 - categorical_accuracy: 0.2926 - val_loss: 1.7082 - val_categorical_accuracy: 0.1900

Epoch 00001: saving model to model_TL_Mobilenet_2020-08-1705_57_31.761419/model-00001-1.57069-0.29261-1.70817-0.19000.h5
Epoch 2/20
83/83 [==============================] - 39s 465ms/step - loss: 1.3235 - categorical_accuracy: 0.4615 - val_loss: 1.6895 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_TL_Mobilenet_2020-08-1705_57_31.761419/model-00002-1.32372-0.46154-1.68949-0.25000.h5
Epoch 3/20
83/83 [==============================] - 42s 503ms/step - loss: 1.1338 - categorical_accuracy: 0.5538 - val_loss: 1.8454 - val_categorical_accuracy: 0.2200

Epoch 00003: saving model to model_TL_Mobilenet_2020-08-1705_57_31.761419/model-00003-1.13434-0.55354-1.84543-0.22000.h5
Epoch 4/20
83/83 [==============================] - 41s 497ms/step - loss: 0.9861 - categorical_accuracy: 0.6091 - val_loss: 1.7921 - val_catego